**LOAD THE PDF**

In [96]:
from langchain_community.document_loaders import PDFMinerLoader
loader = PDFMinerLoader(
    "ALOK-RAI--22BEC010.pdf",
    mode="single",
)
docs = loader.load()
print(docs)


[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-07-04T09:54:40+00:00', 'author': '', 'keywords': '', 'moddate': '2025-07-04T09:54:40+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': 'False', 'total_pages': 1, 'source': 'ALOK-RAI--22BEC010.pdf'}, page_content='ALOK RAI\nMobile: +91-9151348969\n\nEducation\n\nEmail: 22bec010@iiitdmj.ac.in\nLinkedIn: www.linkedin.com/in/alok-rai158\nGitHub: github.com/ALOK158\n\n• Indian Institute of Information Technology, Design and Manufacturing\n\nBachelor of Technology - Electronics and Communication; CPI: 7.9\n\n• Sant Atulanand Convent School\n\n12th - 93.3%\nExperience\n\n• Summer Intern\n\nMachine Learning Intern\n\nJabalpur, India\nNov 2022 - July 2026\n\nVaranasi, India\nApril 2020 - July 2021\n\nProfessor Aparajita Ojha\nMay 2025 - Ongoing\n\n◦ Cancer Classification Using Gene Expr

**PAGE CONTENT**

In [97]:
page_contents = [doc.page_content for doc in docs]
print(page_contents)


['ALOK RAI\nMobile: +91-9151348969\n\nEducation\n\nEmail: 22bec010@iiitdmj.ac.in\nLinkedIn: www.linkedin.com/in/alok-rai158\nGitHub: github.com/ALOK158\n\n• Indian Institute of Information Technology, Design and Manufacturing\n\nBachelor of Technology - Electronics and Communication; CPI: 7.9\n\n• Sant Atulanand Convent School\n\n12th - 93.3%\nExperience\n\n• Summer Intern\n\nMachine Learning Intern\n\nJabalpur, India\nNov 2022 - July 2026\n\nVaranasi, India\nApril 2020 - July 2021\n\nProfessor Aparajita Ojha\nMay 2025 - Ongoing\n\n◦ Cancer Classification Using Gene Expression Data: Utilising a GenAI, Vision Models and DeepInsight framework\n\nto develop a pipeline for Cancer prediction from gene expression data.\n\n• Research Intern\n\nDr. Deep Prakash Samjdar\nOct 2024 - May 2025\n◦ Performance Analysis of Sn-based Perovskite Materials: Built ML models to predict solar cell metrics (Voc, Jsc,\n\nData Analyst Intern\n\nPCE, FF) of 11 materials, conducted EDA to uncover performance tre

In [98]:
# Assuming docs is a list of Document objects
page_contents = [doc.page_content for doc in docs]

# Join all page contents into one complete text
complete_text = " ".join(page_contents)

print(complete_text)


ALOK RAI
Mobile: +91-9151348969

Education

Email: 22bec010@iiitdmj.ac.in
LinkedIn: www.linkedin.com/in/alok-rai158
GitHub: github.com/ALOK158

• Indian Institute of Information Technology, Design and Manufacturing

Bachelor of Technology - Electronics and Communication; CPI: 7.9

• Sant Atulanand Convent School

12th - 93.3%
Experience

• Summer Intern

Machine Learning Intern

Jabalpur, India
Nov 2022 - July 2026

Varanasi, India
April 2020 - July 2021

Professor Aparajita Ojha
May 2025 - Ongoing

◦ Cancer Classification Using Gene Expression Data: Utilising a GenAI, Vision Models and DeepInsight framework

to develop a pipeline for Cancer prediction from gene expression data.

• Research Intern

Dr. Deep Prakash Samjdar
Oct 2024 - May 2025
◦ Performance Analysis of Sn-based Perovskite Materials: Built ML models to predict solar cell metrics (Voc, Jsc,

Data Analyst Intern

PCE, FF) of 11 materials, conducted EDA to uncover performance trends supporting next-gen solar cell research.


In [99]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker



In [100]:
import os
from dotenv import load_dotenv
load_dotenv()

# Load embeddings using Gemini Embedding model
embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# SemanticChunker splits based on semantic similarity
splitter = SemanticChunker(embeddings=embedding_model, breakpoint_threshold_type="percentile")

# Perform semantic splitting
chunks = splitter.split_text(complete_text)

for i, chunk in enumerate(chunks):
    print(f"\n--- Chunk {i+1} ---\n{chunk}")



--- Chunk 1 ---
ALOK RAI
Mobile: +91-9151348969

Education

Email: 22bec010@iiitdmj.ac.in
LinkedIn: www.linkedin.com/in/alok-rai158
GitHub: github.com/ALOK158

• Indian Institute of Information Technology, Design and Manufacturing

Bachelor of Technology - Electronics and Communication; CPI: 7.9

• Sant Atulanand Convent School

12th - 93.3%
Experience

• Summer Intern

Machine Learning Intern

Jabalpur, India
Nov 2022 - July 2026

Varanasi, India
April 2020 - July 2021

Professor Aparajita Ojha
May 2025 - Ongoing

◦ Cancer Classification Using Gene Expression Data: Utilising a GenAI, Vision Models and DeepInsight framework

to develop a pipeline for Cancer prediction from gene expression data. • Research Intern

Dr.

--- Chunk 2 ---
Deep Prakash Samjdar
Oct 2024 - May 2025
◦ Performance Analysis of Sn-based Perovskite Materials: Built ML models to predict solar cell metrics (Voc, Jsc,

Data Analyst Intern

PCE, FF) of 11 materials, conducted EDA to uncover performance trends supporti

**Text-Splitting AND Vectorizing it**

In [101]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
texts = text_splitter.split_text(complete_text)
print(len(texts))

7


In [102]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
import os
import getpass

# embedding_model is already defined above, so you can reuse it
vector_store = FAISS.from_texts(texts, embedding_model)
vector_store.index_to_docstore_id

{0: '4f4e440d-810a-4c47-88bb-82b16e18b4cd',
 1: 'd3777771-f3f5-4185-8a57-f464572bd3b4',
 2: 'bf72be2e-424e-4640-bdfa-f7bc6c977aa6',
 3: '804ecac8-c364-4336-a14c-e89c357cb456',
 4: 'a21f01de-3ff8-4f4c-a503-866496740aa4',
 5: '94c83bfd-edb3-4560-b0bb-c675f149924a',
 6: 'cb959af0-c4a0-4252-b372-132fde5ca828'}

In [103]:
vector_store.get_by_ids(['820eab96-c96f-4291-9232-70d04b2c7414'])

[]

In [105]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("You are an resume helper that helps how well a job desciption is. For now based on job {desciption}, help to derive prompt to extarct all necessary information from retriver ")

prompt_template.invoke({"desciption": '''About the job
Internship Summary:

We're looking for enthusiastic and talented AI Interns passionate about coding to join our team. This internship offers a unique opportunity to gain hands-on experience by working on real-world projects, contributing to innovative solutions. You will play a crucial role in developing advanced AI solutions from data preparation to model deployment.


What You will Do:

As an AI Engineer Intern, you'll work closely with our team, contributing to various stages of the AI development lifecycle. Your responsibilities may include:
Data Preparation & Annotation: Assist in collecting, cleaning, preprocessing, and annotating large datasets This will involve supporting robust OCR and HTR capabilities.
Model Experimentation & Evaluation: Conduct experiments to test, evaluate, and fine-tune AI models for accuracy, performance, and scalability.
Information Extraction & Analysis: Contribute to the development and testing of features for highly accurate multilingual information extraction, retrieval, summarization, and enhanced pattern recognition from diverse, unstructured government documents.
Large Language Model (LLM) Support: Support the team in evaluating, fine-tuning, and testing open-source LLMs for performance and suitability, ensuring they deeply understand government-specific terminology, policies, and contextual nuances in both English and Indian languages. 
Research & Exploration: Research the latest advancements, architectures, and techniques
Prototyping & Implementation: Help in building prototypes and integrating AI components, including those for conceptual predictive insights from forecasting models.
Experiment with prompt engineering techniques to optimize outputs from generative models.
Explore and implement techniques for controlling generated content and ensuring model safety/alignment.
Documentation: Maintain clear and concise documentation of experiments, model development processes, and results.
Collaboration: Actively participate in team discussions, brainstorm new ideas, and collaborate with cross-functional teams to align AI solutions with business objectives.
 
Who Can Apply:

Currently pursuing or recently completed a Bachelor's or Master's degree in Computer Science, Data Science, Artificial Intelligence, Machine Learning from a Tier 1 or Tier-2 Colleges / Autonomous Institutions.
Strong foundational understanding of core AI and Machine Learning concepts and algorithms.
Specific interest in Natural Language Processing (NLP) and Large Language Models (LLMs) is highly preferred.
Proficiency in at least one programming language, preferably Python is mandatory
Familiarity with deep learning frameworks (e.g., TensorFlow) is a plus.
Excellent analytical and problem-solving skills, with a keen eye for detail in visual data and model outputs.
Strong communication and teamwork skills, with the ability to articulate technical concepts clearly.
A strong desire to learn, adapt, and contribute in a fast-paced environment focused on innovation.'''})

StringPromptValue(text="You are an resume helper that helps how well a job desciption is. For now based on job About the job\nInternship Summary:\n\nWe're looking for enthusiastic and talented AI Interns passionate about coding to join our team. This internship offers a unique opportunity to gain hands-on experience by working on real-world projects, contributing to innovative solutions. You will play a crucial role in developing advanced AI solutions from data preparation to model deployment.\n\n\nWhat You will Do:\n\nAs an AI Engineer Intern, you'll work closely with our team, contributing to various stages of the AI development lifecycle. Your responsibilities may include:\nData Preparation & Annotation: Assist in collecting, cleaning, preprocessing, and annotating large datasets This will involve supporting robust OCR and HTR capabilities.\nModel Experimentation & Evaluation: Conduct experiments to test, evaluate, and fine-tune AI models for accuracy, performance, and scalability.\

In [106]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers.json import SimpleJsonOutputParser

# ---------- 1. Prompt ----------
job_jd_extraction_template = """
You are a resume-assistant that extracts structured data from job descriptions.

Return ONLY valid JSON with these keys:
- title                       (string or null)
- company                     (string or null)
- location                    (string or null)
- summary                     (string)
- responsibilities            (string[])
- required_skills             (string[])
- preferred_skills            (string[])
- education_requirements      (string or null)
- experience_level            (string or null)
- Else                       (string or null)

If a field is missing, use null or an empty list. In description, there can be multiple paragraphs, so you should extract the relevant information from the entire text and sometimes keys for JSON file will have synonyms or different names, so you should extract the relevant information from the entire text and sometimes keys for JSON file will have synonyms or different names.

Job description:
\"\"\"{description}\"\"\"
"""

prompt = PromptTemplate(
    input_variables=["description"],
    template=job_jd_extraction_template.strip()
)

# ---------- 2. LLM ----------
# Gemini-Pro is the general-purpose chat model.
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",      # <- or "gemini-pro-vision" etc.
    temperature=0.0          # deterministic extraction
)

# ---------- 3. Chain ----------
chain = prompt | llm | SimpleJsonOutputParser()

# ---------- 4. Run ----------
jd = """ About the job
Internship Summary:

We're looking for enthusiastic and talented AI Interns passionate about coding to join our team. This internship offers a unique opportunity to gain hands-on experience by working on real-world projects, contributing to innovative solutions. You will play a crucial role in developing advanced AI solutions from data preparation to model deployment.


What You will Do:

As an AI Engineer Intern, you'll work closely with our team, contributing to various stages of the AI development lifecycle. Your responsibilities may include:
Data Preparation & Annotation: Assist in collecting, cleaning, preprocessing, and annotating large datasets This will involve supporting robust OCR and HTR capabilities.
Model Experimentation & Evaluation: Conduct experiments to test, evaluate, and fine-tune AI models for accuracy, performance, and scalability.
Information Extraction & Analysis: Contribute to the development and testing of features for highly accurate multilingual information extraction, retrieval, summarization, and enhanced pattern recognition from diverse, unstructured government documents.
Large Language Model (LLM) Support: Support the team in evaluating, fine-tuning, and testing open-source LLMs for performance and suitability, ensuring they deeply understand government-specific terminology, policies, and contextual nuances in both English and Indian languages. 
Research & Exploration: Research the latest advancements, architectures, and techniques
Prototyping & Implementation: Help in building prototypes and integrating AI components, including those for conceptual predictive insights from forecasting models.
Experiment with prompt engineering techniques to optimize outputs from generative models.
Explore and implement techniques for controlling generated content and ensuring model safety/alignment.
Documentation: Maintain clear and concise documentation of experiments, model development processes, and results.
Collaboration: Actively participate in team discussions, brainstorm new ideas, and collaborate with cross-functional teams to align AI solutions with business objectives.
 
Who Can Apply:

Currently pursuing or recently completed a Bachelor's or Master's degree in Computer Science, Data Science, Artificial Intelligence, Machine Learning from a Tier 1 or Tier-2 Colleges / Autonomous Institutions.
Strong foundational understanding of core AI and Machine Learning concepts and algorithms.
Specific interest in Natural Language Processing (NLP) and Large Language Models (LLMs) is highly preferred.
Proficiency in at least one programming language, preferably Python is mandatory
Familiarity with deep learning frameworks (e.g., TensorFlow) is a plus.
Excellent analytical and problem-solving skills, with a keen eye for detail in visual data and model outputs.
Strong communication and teamwork skills, with the ability to articulate technical concepts clearly.
A strong desire to learn, adapt, and contribute in a fast-paced environment focused on innovation.
 """
result = chain.invoke({"description": jd})
print(result)


{'title': 'AI Intern', 'company': None, 'location': None, 'summary': "We're looking for enthusiastic and talented AI Interns passionate about coding to join our team. This internship offers a unique opportunity to gain hands-on experience by working on real-world projects, contributing to innovative solutions. You will play a crucial role in developing advanced AI solutions from data preparation to model deployment.", 'responsibilities': ['Data Preparation & Annotation: Assist in collecting, cleaning, preprocessing, and annotating large datasets. This will involve supporting robust OCR and HTR capabilities.', 'Model Experimentation & Evaluation: Conduct experiments to test, evaluate, and fine-tune AI models for accuracy, performance, and scalability.', 'Information Extraction & Analysis: Contribute to the development and testing of features for highly accurate multilingual information extraction, retrieval, summarization, and enhanced pattern recognition from diverse, unstructured gove

In [107]:
result

{'title': 'AI Intern',
 'company': None,
 'location': None,
 'summary': "We're looking for enthusiastic and talented AI Interns passionate about coding to join our team. This internship offers a unique opportunity to gain hands-on experience by working on real-world projects, contributing to innovative solutions. You will play a crucial role in developing advanced AI solutions from data preparation to model deployment.",
 'responsibilities': ['Data Preparation & Annotation: Assist in collecting, cleaning, preprocessing, and annotating large datasets. This will involve supporting robust OCR and HTR capabilities.',
  'Model Experimentation & Evaluation: Conduct experiments to test, evaluate, and fine-tune AI models for accuracy, performance, and scalability.',
  'Information Extraction & Analysis: Contribute to the development and testing of features for highly accurate multilingual information extraction, retrieval, summarization, and enhanced pattern recognition from diverse, unstructu

In [108]:
retriever.invoke("Now based on {result} and  the stuff from resume extract all necessary information from retriver retrive the information from retriver ")

[Document(id='804ecac8-c364-4336-a14c-e89c357cb456', metadata={}, page_content='Skills and Competencies\n\n3.1 Skills Summary\n\nProgramming Languages:Python(3+yrs), C++(3+ yrs), SQL, Matlab\nML/NLP Frameworks: TensorFlow, PyTorch, Pandas, Scikit-learn, RAGs\nVS Code, Google Colab, Hugging Face, LangChain\nTools:\nPower BI, Streamlit(2+yrs), Cadence(2+yrs), Arduino\nOther:\n\nProjects\n\n• Transformer Language Model:\n\n◦ Implemented a GPT-style transformer-2.2 million parameters from scratch in PyTorch for autoregressive text\n\ngeneration.\n\n◦ Designed with Multi-head self-attention (6 heads), residual connections, and achieving a validation perplexity approx\n\n10 over 5000 iterations.'),
 Document(id='bf72be2e-424e-4640-bdfa-f7bc6c977aa6', metadata={}, page_content='• Summer Intern\n\nMachine Learning Intern\n\nIIITDM Jabalpur\nMay 2024 - July 2024\n\n◦ Sign Language Detection Model: Developed a Convolutional Neural Network (CNN) model for real-time sign\n\nlanguage detection usin

**MULTI-QUERY-RETIRVER**

In [109]:
def build_search_query(job: dict) -> str:
    """
    Turn a structured job-post into a single keyword string.

    Parameters
    ----------
    job : dict
        Must contain lists under some or all of these keys:
        - "responsibilities"
        - "required_skills"
        - "preferred_skills"
        Everything else is ignored.

    Returns
    -------
    str
        Comma-separated list with duplicates removed, e.g.
        "Python, TensorFlow, Data preparation, NLP, LLM fine-tuning"
    """
    # Collect lists; if the key is missing fall back to an empty list
    resp    = job.get("responsibilities", [])
    req     = job.get("required_skills", [])
    pref    = job.get("preferred_skills", [])

    # Flatten + deduplicate while preserving order
    seen, parts = set(), []
    for item in resp + req + pref:
        if isinstance(item, str) and item not in seen:
            seen.add(item)
            parts.append(item)

    return ", ".join(parts)


In [110]:
# --- 0. prerequisites ---------------------------------------------------------
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

# --- 1. base retriever --------------------------------------------------------
# pull (for example) the 2 most similar chunks for *each* query
base_retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 7}
)
from langchain_google_genai import ChatGoogleGenerativeAI   # make sure this import matches your package version

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",   # <- correct kwarg
    temperature=0.2,
)


# --- 3. wrap in a Multi-Query retriever --------------------------------------
multi_query_retriever = MultiQueryRetriever.from_llm(
        retriever   = base_retriever,   # your existing retriever
        llm         = llm
)

# 4. build search query from job post
search_query = build_search_query(result)

docs =multi_query_retriever.get_relevant_documents(search_query)

# --- 4. use it like any other retriever --------------------------------------

print(len(docs), docs[0].page_content)
print(docs)

# or plug it straight into a QA / conversation chain
# from langchain.chains import ConversationalRetrievalChain
# qa_chain = ConversationalRetrievalChain.from_llm(llm, multi_query_retriever)


7 Skills and Competencies

3.1 Skills Summary

Programming Languages:Python(3+yrs), C++(3+ yrs), SQL, Matlab
ML/NLP Frameworks: TensorFlow, PyTorch, Pandas, Scikit-learn, RAGs
VS Code, Google Colab, Hugging Face, LangChain
Tools:
Power BI, Streamlit(2+yrs), Cadence(2+yrs), Arduino
Other:

Projects

• Transformer Language Model:

◦ Implemented a GPT-style transformer-2.2 million parameters from scratch in PyTorch for autoregressive text

generation.

◦ Designed with Multi-head self-attention (6 heads), residual connections, and achieving a validation perplexity approx

10 over 5000 iterations.
[Document(id='804ecac8-c364-4336-a14c-e89c357cb456', metadata={}, page_content='Skills and Competencies\n\n3.1 Skills Summary\n\nProgramming Languages:Python(3+yrs), C++(3+ yrs), SQL, Matlab\nML/NLP Frameworks: TensorFlow, PyTorch, Pandas, Scikit-learn, RAGs\nVS Code, Google Colab, Hugging Face, LangChain\nTools:\nPower BI, Streamlit(2+yrs), Cadence(2+yrs), Arduino\nOther:\n\nProjects\n\n• Transfo

**PROMPT TO COMPARE JOB DESCRIPTION AND THE RESUME**

In [111]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# 1-- LLM instance
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# 2-- prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a career-matching assistant. "
     "Use ONLY the resume snippets provided to judge the fit."),
    ("user", "Job description (JSON):\n{job_json}"),
    ("user", "Resume snippets:\n{snippets}"),
    ("user",
     "Return a JSON with keys:\n"
     "  - matches  (bullet points where the resume meets the job)\n"
     "  - gaps     (bullet points where the resume is missing something)\n"
     "  - overall_fit (one-sentence conclusion)-- be very strict in your assessment!"),
])

# 3-- combine template + LLM  -> runnable chain
chain = prompt | llm          # equivalent to RunnableSequence(prompt, llm)

# 4-- variables you collected earlier
variables = {
    "job_json": result,          # dict or JSON string
    "snippets": docs                # "\n---\n".join(d.page_content for d in docs)
}

# 5-- run the chain
answer = chain.invoke(variables)        # returns an LLM message object
print(answer.content)                   # or just `print(answer)` depending on version


```json
{
  "matches": [
    "The resume shows proficiency in Python and experience with TensorFlow, both required skills.",
    "The candidate has experience with NLP, including building a transformer language model and working on a food classification model, aligning with the preferred skills and responsibilities.",
    "The resume demonstrates experience in model development, evaluation, and optimization, including hyperparameter tuning and techniques to reduce computation time.",
    "The candidate's projects showcase experience in various aspects of AI development, such as data preprocessing, model training, and performance analysis.",
    "The candidate's internship experience at IIITDM Jabalpur aligns with the required education level."
  ],
  "gaps": [
    "The resume does not explicitly mention experience with large language models (LLMs) beyond building a transformer model, which is a key preferred skill and responsibility.",
    "There is no mention of experience with multil

**CHAINING UP THE WHOLE PROCESS**

In [113]:
# 5. comparison prompt
comparison = prompt | llm
result = comparison.invoke({"job_json": result,
                            "snippets": "\n---\n".join(d.page_content for d in docs)})
print(result)

content='```json\n{\n  "matches": [\n    "The resume demonstrates experience with TensorFlow and PyTorch, as well as other relevant tools like Hugging Face and LangChain.",\n    "The candidate has multiple projects showcasing experience in model development and optimization, including a Transformer Language Model and a food classification model.",\n    "The resume indicates experience with NLP through projects like the Transformer Language Model and EZE-Paragrapher."\n  ],\n  "gaps": [\n    "The resume does not mention experience with data preparation and annotation.",\n    "There is no mention of experience with multilingual information extraction or working with government documents.",\n    "The resume lacks explicit demonstration of experience with LLMs beyond general NLP experience.",\n    "The candidate\'s education is not explicitly stated as being from a Tier 1 or Tier 2 college.",\n    "The resume does not highlight experience with prompt engineering or model safety/alignment t